# Jupyter pglite anywidget demo

Demo of using `jupyterlite_anywidget_pglite` widget and magics.

Works in:

- Jupyter UIs (JupyterLab, Jupyter notebook)
- VS Code (via Jupyter notebook support)

## Headless Demo

In the headless demo, no HTML UI elements are added to the DOM, but the `anywidget` machinery *is* added via a call to `IPython.display.display`.

In [1]:
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_headless

In [2]:
pg_headless = pglite_headless()

postgresWidget(headless=True, response={'status': 'initialising'})

In [3]:
%%pglite_magic -w pg_headless
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null
      );

In [4]:
%%pglite_magic -w pg_headless
INSERT INTO test (title) VALUES ('dummy_headless');


In [5]:
%%pglite_magic -w pg_headless
SELECT * FROM test;

In [6]:
pg_headless.response

{'status': 'processing'}

Use the `.df()` method on the widget to display a query result as a `pandas` dafaframe, if pandas is installed.

In [7]:
pg_headless.df()

KeyError: 'response'

## Database persistence

We can persist the database using a browser's `IndexedDB` datastore, passing the name of the indexed database via an `idb=` parameter (Jupyter browser-based UIs; does not work in VSCode).

This parameter can also be used in the inline and panel display database creation steps.

In [ ]:
pg_headless_persist = pglite_headless(idb="pglitetest1")

In [8]:
# Close / shutdown

AttributeError: 'function' object has no attribute 'close'

## Inline HTML Display (Initialising cell)

We can display the widget inline as a cell output, althoug the utility of this is perhaps a little bit limited?

In [1]:
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_inline

pg_inline = pglite_inline()

postgresWidget(response={'status': 'initialising'})

In [7]:
%%pglite_magic -w pg_inline
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null
      );

In [8]:
%%pglite_magic
INSERT INTO test (title) VALUES ('dummy_inline');

In [8]:
%pglite_query -D -r -q 'SELECT * FROM test LIMIT 1;'

UsageError: unrecognized arguments: -D


In [13]:
pg_inline.response

{'rows': [{'id': 1, 'title': 'dummy'}],
 'fields': [{'name': 'id', 'dataTypeID': 23},
  {'name': 'title', 'dataTypeID': 1043}],
 'affectedRows': 0}

## HTML in own panel

In a Juoyter Lab environment, we can use [`jupyterlab-sidecar`](https://github.com/jupyter-widgets/jupyterlab-sidecar) to display the widget HTML UI in its own panel (note: this does not currently work at all in VS Code).

In [1]:
# Load in the widget panel
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_panel

# Launch it
pg_panel = pglite_panel()
# Wait for it to be ready
pg_panel.ready()

In [2]:
# About the db
pg_panel.about

{'version': 'PostgreSQL 16.4 on x86_64-pc-linux-gnu, compiled by emcc (Emscripten gcc/clang-like replacement + linker emulating GNU ld) 3.1.72 (437140d149d9c977ffc8b09dbaf9b0f5a02db190), 32-bit'}

In [3]:
%%pglite_magic -w pg_panel
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null
)

In [4]:
# Response from db
pg_panel.response

{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 0},
 'response_type': 'single'}

In [5]:
%%pglite_magic -r
SELECT * FROM test;

{'status': 'completed',
 'response': {'rows': [],
  'fields': [{'name': 'id', 'dataTypeID': 23},
   {'name': 'title', 'dataTypeID': 1043}],
  'affectedRows': 0},
 'response_type': 'single'}

In [6]:
%%pglite_magic
INSERT INTO test (title) VALUES ('dummy_panel');

In [ ]:
%pglite_query -t 15 -r -D -q 'SELECT * FROM test LIMIT 1;'

In [6]:
%%pglite_magic -m
INSERT INTO test (title) VALUES ('dummy_panel1');
INSERT INTO test (title) VALUES ('dummy_panel2');

In [7]:
%%pglite_magic -M
INSERT INTO test (title) VALUES ('dummy_panel3');
INSERT INTO test (title) VALUES ('dummy_panel4');

In [8]:
%%pglite_magic -r -d
SELECT * FROM test;

,title
id,
1,dummy_panel1
2,dummy_panel2
3,dummy_panel3
4,dummy_panel4


In [9]:
pg_panel.df()

,title
id,
1,dummy_panel1
2,dummy_panel2
3,dummy_panel3
4,dummy_panel4


In [10]:
pg_panel.response

{'status': 'completed',
 'response': {'rows': [{'id': 1, 'title': 'dummy_panel1'},
   {'id': 2, 'title': 'dummy_panel2'},
   {'id': 3, 'title': 'dummy_panel3'},
   {'id': 4, 'title': 'dummy_panel4'}],
  'fields': [{'name': 'id', 'dataTypeID': 23},
   {'name': 'title', 'dataTypeID': 1043}],
  'affectedRows': 0},
 'response_type': 'single'}

In [11]:
df = pg_panel.df()
%pglite_df_insert -d df -t test

In [12]:
%%pglite_magic -r -d
SELECT * FROM test;

,title
id,
1,dummy_panel1
2,dummy_panel2
3,dummy_panel3
4,dummy_panel4
5,dummy_panel1
6,dummy_panel2
7,dummy_panel3
8,dummy_panel4


In [9]:
pg_panel.close()